In [55]:
%matplotlib inline
import pandas as pd 
import numpy as np
import ast
import warnings; warnings.simplefilter('ignore')

In [62]:
movie_data = pd.read_csv("movies_metadata.csv", low_memory = False)
movie_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [49]:
movie_data["genres"]

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [63]:
movie_data["genres"]= movie_data["genres"].fillna("[]").apply(ast.literal_eval).apply(lambda x: [i["name"] for i in x] if isinstance(x,list) else [])
movie_data["genres"]

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

In [27]:


#movie_data['genres'] = movie_data['genres'].apply(lambda x: [i['name'] for i in x] if isinstance in (x,list) else ["name"]) 
#movie_data['genres']

0        [name]
1        [name]
2        [name]
3        [name]
4        [name]
          ...  
45461    [name]
45462    [name]
45463    [name]
45464    [name]
45465    [name]
Name: genres, Length: 45466, dtype: object

In [64]:
vote_count = movie_data[movie_data["vote_count"].notnull()]["vote_count"].astype("int")
vote_count

0        5415
1        2413
2          92
3          34
4         173
         ... 
45461       1
45462       3
45463       6
45464       0
45465       0
Name: vote_count, Length: 45460, dtype: int64

In [65]:
vote_average = movie_data[movie_data["vote_average"].notnull()]["vote_average"].astype("int")
vote_average
c= vote_average.mean()
c

5.244896612406511

In [66]:
m = vote_count.quantile(0.95)
m

434.0

In [67]:
movie_data["year"] = pd.to_datetime(movie_data["release_date"], errors = 'coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [68]:
qualified = movie_data[(movie_data["vote_count"]>= m) & movie_data["vote_count"].notnull() & (movie_data["vote_average"].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count']= qualified["vote_count"].astype(int)
qualified['vote_average']=qualified["vote_average"].astype(int)
qualified.shape

(2274, 6)

In [69]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * c)

In [70]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)


In [71]:
qualified["genres"]

0                              [Animation, Comedy, Family]
1                             [Adventure, Fantasy, Family]
5                         [Action, Crime, Drama, Thriller]
9                            [Adventure, Action, Thriller]
15                                          [Drama, Crime]
                               ...                        
44624                          [Science Fiction, Thriller]
44632                                   [Action, Thriller]
44678              [Action, Drama, History, Thriller, War]
44842       [Action, Science Fiction, Thriller, Adventure]
45014    [Action, Western, Science Fiction, Fantasy, Ho...
Name: genres, Length: 2274, dtype: object

In [84]:
s = movie_data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = movie_data.drop('genres', axis=1).join(s)
s

0        Animation
0           Comedy
0           Family
1        Adventure
1          Fantasy
           ...    
45461       Family
45462        Drama
45463       Action
45463        Drama
45463     Thriller
Name: genre, Length: 91106, dtype: object

In [83]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * c), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [82]:
build_chart('Animation').head(15)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.909339
5481,Spirited Away,2001,3968,8,41.048867,7.875933
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.772103
2884,Princess Mononoke,1997,2041,8,17.166725,7.771305
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.735274
40251,Your Name.,2016,1030,8,34.461252,7.589820
5553,Grave of the Fireflies,1988,974,8,0.010902,7.570962
19901,Paperman,2012,734,8,7.198633,7.465676
39386,Piper,2016,487,8,11.243161,7.285132
20779,Wolf Children,2012,483,8,10.249498,7.281198
